In [21]:
%cd  /home/wanddy/ChatGLM-Tuning/
import os
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "max_split_size_mb:32"
os.environ["CUDA_VISIBLE_DEVICES"] = "3"

/home/wanddy/ChatGLM-Tuning


In [22]:

from modeling_chatglm import ChatGLMForConditionalGeneration
import torch


#torch.set_default_tensor_type(torch.cuda.FloatTensor)
model = ChatGLMForConditionalGeneration.from_pretrained("chatglm-6b", trust_remote_code=True, device_map='auto')

The argument `trust_remote_code` is to be used with Auto classes. It has no effect here and is ignored.
Loading checkpoint shards: 100%|██████████| 8/8 [00:08<00:00,  1.10s/it]


In [23]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("chatglm-6b", trust_remote_code=True)

Explicitly passing a `revision` is encouraged when loading a model with custom code to ensure no malicious code has been contributed in a newer revision.


In [24]:
from peft import get_peft_model, LoraConfig, TaskType

peft_path = "output/adapter_model.bin"

peft_config = LoraConfig(
    task_type=TaskType.CAUSAL_LM, inference_mode=True,
    r=8,
    lora_alpha=32, lora_dropout=0.1
)

model = get_peft_model(model, peft_config)
model.load_state_dict(torch.load(peft_path), strict=False)
torch.set_default_tensor_type(torch.cuda.FloatTensor)

/home/wanddy/ChatGLM-6B/.conda/lib/python3.9/site-packages/peft/tuners/lora.py:175: UserWarning: fan_in_fan_out is set to True but the target module is not a Conv1D. Setting fan_in_fan_out to False.
  warnings.warn(


In [25]:
import json

instructions = json.load(open("data/alpaca_data.json",encoding="utf-8"))

In [26]:
answers = []
from cover_alpaca2jsonl import format_example


with torch.no_grad():
    for idx, item in enumerate(instructions[:5]):
        feature = format_example(item)
        input_text = feature['context']
        ids = tokenizer.encode(input_text)
        input_ids = torch.LongTensor([ids])
        out = model.generate(
            input_ids=input_ids,
            max_length=150,
            do_sample=False,
            temperature=0
        )
        out_text = tokenizer.decode(out[0])
        answer = out_text.replace(input_text, "").replace("\nEND", "").strip()
        item['infer_answer'] = answer
        print(out_text)
        print(f"### {idx+1}.Answer:\n", item.get('output'), '\n\n')
        answers.append({'index': idx, **item})

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:11                                                                                   │
│                                                                                                  │
│    8 │   │   input_text = feature['context']                                                     │
│    9 │   │   ids = tokenizer.encode(input_text)                                                  │
│   10 │   │   input_ids = torch.LongTensor([ids])                                                 │
│ ❱ 11 │   │   out = model.generate(                                                               │
│   12 │   │   │   input_ids=input_ids,                                                            │
│   13 │   │   │   max_length=150,                                                                 │
│   14 │   │   │   do_sample=False,                                                                │
│                                                                                                  │
│ /home/wanddy/ChatGLM-6B/.conda/lib/python3.9/site-packages/peft/peft_model.py:581 in generate    │
│                                                                                                  │
│   578 │   │   self.base_model.prepare_inputs_for_generation = self.prepare_inputs_for_generati   │
│   579 │   │   try:                                                                               │
│   580 │   │   │   if not isinstance(self.peft_config, PromptLearningConfig):                     │
│ ❱ 581 │   │   │   │   outputs = self.base_model.generate(**kwargs)                               │
│   582 │   │   │   else:                                                                          │
│   583 │   │   │   │   if "input_ids" not in kwargs:                                              │
│   584 │   │   │   │   │   raise ValueError("input_ids must be provided for Peft model generati   │
│                                                                                                  │
│ /home/wanddy/ChatGLM-6B/.conda/lib/python3.9/site-packages/torch/autograd/grad_mode.py:27 in     │
│ decorate_context                                                                                 │
│                                                                                                  │
│    24 │   │   @functools.wraps(func)                                                             │
│    25 │   │   def decorate_context(*args, **kwargs):                                             │
│    26 │   │   │   with self.clone():                                                             │
│ ❱  27 │   │   │   │   return func(*args, **kwargs)                                               │
│    28 │   │   return cast(F, decorate_context)                                                   │
│    29 │                                                                                          │
│    30 │   def _wrap_generator(self, func):                                                       │
│                                                                                                  │
│ /home/wanddy/ChatGLM-6B/.conda/lib/python3.9/site-packages/transformers/generation/utils.py:1406 │
│ in generate                                                                                      │
│                                                                                                  │
│   1403 │   │   │   │   )                                                                         │
│   1404 │   │   │                                                                                 │
│   1405 │   │   │   # 11. run greedy search                                                       │
│ ❱ 1406 │   │   │   return self.greedy_search(                                                    │
│   1407 │   │   │   │   input_ids,                          